In [1]:
import numpy as np

In [3]:
def abs_rel_error(img1, img2):
  """
  Calculates the absolute relative error between two images.

  Args:
    img1: The first image as a NumPy array.
    img2: The second image as a NumPy array.

  Returns:
    The absolute relative error between the two images.
  """

  assert img1.shape == img2.shape, "Images must have the same shape"

  diff = np.abs(img1 - img2)
  rel_diff = diff / (img2 + 1e-8)  # Add a small value to avoid division by zero
  abs_rel = np.mean(rel_diff)
  return abs_rel

## least squares alignment